This is model, where we should do binary classification - predict probability of A

   1) num_classes - number of classes
   
   2) each class has its popability of A
   
   3) prob - array of these probabilities; they are Q - coordinates for Hamiltonian MCMC
   
   4) prior (alpha, beta) are two float variables describing beta-distribution classes' popabilities.

In [1]:
! python --version

Python 3.5.2


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import pandas as pd
import warnings
from matplotlib import pyplot as plt
from matplotlib.cm import rainbow
import collections
from tqdm import tqdm_notebook as tqdm
import scipy.stats as sps

import tensorflow as tf
import tensorflow_probability as tfp
from IPython.display import clear_output

%matplotlib inline


tfd = tfp.distributions


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
def reset_tf():
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        tf.reset_default_graph()
        try:
            sess.close()
        except:
            pass
        sess = tf.InteractiveSession()
        
    return sess

In [4]:
def get_example_data():
    ps_data_arr = np.array([
        [20, 10, 1, 0.499993], [20, 3, 2, 0.230211], [20, 8, 3, 0.236831], 
        [20, 7, 4, 0.246463], [20, 6, 5, 0.370862], [20, 5, 6, 0.320656], 
        [20, 10, 7, 0.519887], [20, 12, 8, 0.52845], [20, 8, 9, 0.453077], 
        [20, 8, 10, 0.431245], [20, 10, 11, 0.499243], [20, 9, 12, 0.471968], 
        [20, 2, 13, 0.152176], [20, 14, 14, 0.48496], [20, 6, 15, 0.246193]
    ])
    ps_data_pd=pd.DataFrame(data=ps_data_arr[0:, 0:],
             index=ps_data_arr[0:, 2],
             columns=["total_count", "clicks", "class_id", "true_p"],
             dtype=np.float32)
    ps_data_pd['class_id'] = ps_data_pd.class_id.astype('int32')
    
    return ps_data_pd

In [5]:
def sample_data(classes=15, alpha=1, beta=10, samples=20):
    p = np.random.beta(alpha, beta, size=classes)
    clicks = np.random.binomial(samples, p)
    
    ps_data_arr = np.array([
        [samples, clicks[i], i + 1, p[i]] for i in range(classes)
    ])
    
    ps_data_pd=pd.DataFrame(data=ps_data_arr[0:, 0:],
             index=ps_data_arr[0:, 2],
             columns=["total_count", "clicks", "class_id", "true_p"],
             dtype=np.float32)
    ps_data_pd['class_id'] = ps_data_pd.class_id.astype('int32')
    
    return ps_data_pd

In [6]:
def init_HMC_graph(ps_data_pd, sample_size, num_burnin_steps, num_steps_between_results, num_leapfrog_steps, adaptation_steps, step_size_initializer):
    sess = reset_tf()

    def _make_ps_prior(num_classes, alpha, beta, dtype):
        return tfd.Independent(
          tfd.Beta(
              alpha * tf.ones(num_classes),
              beta * tf.ones(num_classes)),
              reinterpreted_batch_ndims=1)

    def _make_ps_log_likelihood(prob, class_id, total_count):
        prob_c = tf.gather(prob, indices=tf.to_int32(class_id - 1), axis=-1)
        total_count_c = tf.gather(total_count, indices=tf.to_int32(class_id - 1), axis=-1)
        return tfp.distributions.Binomial(total_count=tf.to_float(total_count_c), probs=prob_c)
    
    make_ps_prior = tf.make_template(name_='make_ps_prior', func_=_make_ps_prior)
    make_ps_log_likelihood = tf.make_template(name_='make_ps_log_likelihood', func_=_make_ps_log_likelihood)
    
    def joint_log_prob(prob, alpha, beta, total_count, clicks, class_id, dtype):
        num_classes = len(total_count)
        rv_prob = make_ps_prior(num_classes, alpha, beta, dtype)
        rv_clicks = make_ps_log_likelihood(prob, class_id, total_count)
        return (rv_prob.log_prob(prob) + 
             tf.reduce_sum(rv_clicks.log_prob(clicks), axis=-1))
    
    dtype = np.float32
    def unnormalized_posterior_log_prob(prob, log_alpha, log_beta):
        return joint_log_prob(
            prob=tf.sigmoid(prob),
            alpha=tf.exp(log_alpha),
            beta=tf.exp(log_beta),
            total_count=dtype(ps_data_pd.total_count.values),
            clicks=dtype(ps_data_pd.clicks.values),
            class_id=np.int32(ps_data_pd.class_id.values),
            dtype=dtype)

    step_size = tf.get_variable(
        'step_size',
        initializer=step_size_initializer,
        trainable=False)
    
    hmc = tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=unnormalized_posterior_log_prob,
        num_leapfrog_steps=num_leapfrog_steps,
        step_size=step_size,
        step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(adaptation_steps)
    )

    parameters_num = len(ps_data_pd)
    init_random_weights = tf.placeholder(dtype, shape=[parameters_num])
    alpha_pl = tf.placeholder(dtype, shape=())
    beta_pl = tf.placeholder(dtype, shape=())
    
    next_states, kernel_results = tfp.mcmc.sample_chain(
        num_results=sample_size,
        num_burnin_steps=num_burnin_steps,
        num_steps_between_results=num_steps_between_results,
        current_state=[init_random_weights, alpha_pl, beta_pl],
        kernel=hmc)
    
    accepted_results = kernel_results.accepted_results
    
    init_op = tf.initialize_all_variables()
    
    init_op.run()
    
    return init_random_weights, alpha_pl, beta_pl, kernel_results, next_states, sess

In [14]:
def HMC_solution(train, val, L):
    ps_data_arr = np.array([
        [L, train[i] + val[i], i + 1, 0]
        for i in range(train.shape[0])
    ])
    ps_data_pd=pd.DataFrame(data=ps_data_arr[0:, 0:],
             index=ps_data_arr[0:, 2],
             columns=["total_count", "clicks", "class_id", "true_p"],
             dtype=np.float32)
    ps_data_pd['class_id'] = ps_data_pd.class_id.astype('int32')
    
    sample_size = 5000
    num_burnin_steps = 0
    num_steps_between_results = 0
    num_leapfrog_steps = 3
    adaptation_steps = None
    step_size_initializer = 0.01
    init_random_weights, alpha_pl, beta_pl, kernel_results, next_states, sess = init_HMC_graph(
        ps_data_pd, sample_size, num_burnin_steps, num_steps_between_results, num_leapfrog_steps, adaptation_steps, step_size_initializer)
    
    parameters_num = len(ps_data_pd)
    dtype = np.float32
    
    w_ = 0.5 * np.ones([parameters_num], dtype=dtype)
    alpha_ = .1
    beta_ = .1
    
    p_prob = np.zeros_like(w_, dtype=np.float32)
    
    alphas = np.array([])
    betas = np.array([])
    
    for i in tqdm(range(100)):

        [
          kernel_results_,
          next_states_
        ] = sess.run([
          kernel_results,
          next_states
        ], feed_dict={init_random_weights: w_, alpha_pl: alpha_, beta_pl: beta_})

        num_accepted = kernel_results_.is_accepted.sum()
        num_drawn = kernel_results_.is_accepted.size
        acceptance_rate = num_accepted / num_drawn
        print (num_accepted, num_drawn, acceptance_rate)
        for i in range(sample_size):
            p_prob += bayesian_solve(next_states_[1][i], next_states_[2][i], train + val, 2 * L)
        #p_prob += (1 / (1 + np.exp(-next_states_[0]))).mean(axis=0) какие то очень плохие результаты
        w_ = next_states_[0][-1]
        alpha_ = next_states_[1][-1]
        beta_ = next_states_[2][-1]
        
        #alphas = np.append(alphas, next_states_[1])
        #betas = np.append(betas, next_states_[2])
    print (p_prob / 100 / sample_size)
    return p_prob / 100 / sample_size

In [15]:
from datasets import *
data_set_small = load_data_set('./datasets/100.10')
data_set_big = load_data_set('./datasets/100.100')

In [16]:
from basic_solutions import *

evaluate(stupid_solution, data_set_big.train_data[0], data_set_big.val_data[0], data_set_big.ideal[0], data_set_big.L)

-0.02085861220032036

In [17]:
t_id = 0


evaluate(HMC_solution,
         data_set_big.train_data[t_id],
         data_set_big.val_data[t_id],
         data_set_big.ideal[t_id],
         data_set_big.L)

2462 5000 0.4924
3019 5000 0.6038
3038 5000 0.6076
3000 5000 0.6
3048 5000 0.6096
3001 5000 0.6002
3014 5000 0.6028
3006 5000 0.6012
2950 5000 0.59
3013 5000 0.6026
3020 5000 0.604
3045 5000 0.609
3042 5000 0.6084
3006 5000 0.6012
3041 5000 0.6082
3046 5000 0.6092
3072 5000 0.6144
2904 5000 0.5808
2929 5000 0.5858
2949 5000 0.5898
2926 5000 0.5852
2912 5000 0.5824
2844 5000 0.5688
2837 5000 0.5674
3013 5000 0.6026
2988 5000 0.5976
2962 5000 0.5924
2874 5000 0.5748
2848 5000 0.5696
2940 5000 0.588
2964 5000 0.5928
2876 5000 0.5752
2899 5000 0.5798
2893 5000 0.5786
2909 5000 0.5818
2858 5000 0.5716
2865 5000 0.573
2918 5000 0.5836
2830 5000 0.566
2948 5000 0.5896
2981 5000 0.5962
3039 5000 0.6078
2997 5000 0.5994
3034 5000 0.6068
2973 5000 0.5946
3027 5000 0.6054
3013 5000 0.6026
3029 5000 0.6058
2992 5000 0.5984
2927 5000 0.5854
2991 5000 0.5982
2968 5000 0.5936
2993 5000 0.5986
2957 5000 0.5914
2964 5000 0.5928
2947 5000 0.5894
3005 5000 0.601
2963 5000 0.5926
3056 5000 0.6112
2958 500

-0.06394917150604032

In [11]:
def sample_alpha_and_beta(ps_data_pd, sample_size, num_burnin_steps, num_steps_between_results, num_leapfrog_steps, adaptation_steps):
    init_random_weights, alpha_pl, beta_pl, kernel_results, next_states, sess = init_HMC_graph(ps_data_pd, sample_size, num_burnin_steps, num_steps_between_results, num_leapfrog_steps, adaptation_steps)
    
    parameters_num = len(ps_data_pd)
    dtype = np.float32
    
    w_ = 0.5 * np.ones([parameters_num], dtype=dtype)
    alpha_ = .1
    beta_ = .1
    
    alphas = np.array([])
    betas = np.array([])
    
    for i in tqdm(range(100)):

        [
          kernel_results_,
          next_states_
        ] = sess.run([
          kernel_results,
          next_states
        ], feed_dict={init_random_weights: w_, alpha_pl: alpha_, beta_pl: beta_})

        num_accepted = kernel_results_.is_accepted.sum()
        num_drawn = kernel_results_.is_accepted.size
        acceptance_rate = num_accepted / num_drawn
        print (num_accepted, num_drawn, acceptance_rate)
        w_ = next_states_[0][-1]
        alpha_ = next_states_[1][-1]
        beta_ = next_states_[2][-1]
        
        alphas = np.append(alphas, next_states_[1])
        betas = np.append(betas, next_states_[2])
    
    return np.array([alphas, betas]).T

In [12]:
alpha = 100
beta = 100
data = sample_data(classes=20, alpha=alpha, beta=beta)
data

,total_count,clicks,class_id,true_p
1.0,20.0,11.0,1,0.505107
2.0,20.0,10.0,2,0.515861
3.0,20.0,5.0,3,0.456393
4.0,20.0,5.0,4,0.425608
5.0,20.0,12.0,5,0.541650
6.0,20.0,9.0,6,0.471204
7.0,20.0,6.0,7,0.503902
8.0,20.0,7.0,8,0.479321
9.0,20.0,9.0,9,0.480249
10.0,20.0,9.0,10,0.422055


In [13]:
samples = sample_alpha_and_beta(data, int(1000 * 100), 0, 0, 10, None)

TypeError: init_HMC_graph() missing 1 required positional argument: 'step_size_initializer'

In [ ]:
def plot(samples):
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plt.plot(samples[:, 0], samples[:, 1], alpha=0.3)
    plt.scatter(samples[:, 0], samples[:, 1], c=rainbow(np.linspace(0, 1, samples.shape[0])))

    plt.subplot(1, 2, 2)
    plt.plot((samples[:, 0] - samples[:, 0].mean()) / samples[:, 0].std())
    plt.plot((samples[:, 1] - samples[:, 1].mean()) / samples[:, 1].std())
    plt.show()

In [ ]:
samples.shape

In [ ]:
plot(np.exp(samples[:-80000:1]))

In [ ]:
x = np.linspace(0, 1, 100)
y = sps.beta(0.16, 0.38).pdf(x)
plt.plot(x, y)

In [ ]:
"""
import pickle

with open('chain.pkl', 'wb') as output:
    pickle.dump(chains, output, pickle.HIGHEST_PROTOCOL)
"""

In [ ]:
import pickle

with open('chain.pkl', 'rb') as input_:
    chains = pickle.load(input_)

In [ ]:
len(chains)

In [ ]:
for ch in chains:
    print (len(ch))

In [ ]:
start = 10000
for ch in chains:
    if len(ch) > start:
        print(len(ch))
        samples = ch[start::300]
        
        plt.figure(figsize=(15, 6))
        plt.subplot(1, 2, 1)
        plt.plot(samples[:, 0], samples[:, 1], alpha=0.3)
        plt.scatter(samples[:, 0], samples[:, 1], c=rainbow(np.linspace(0, 1, samples.shape[0])))

        plt.subplot(1, 2, 2)
        plt.plot((samples[:, 0] - samples[:, 0].mean()) / samples[:, 0].std())
        plt.plot((samples[:, 1] - samples[:, 1].mean()) / samples[:, 1].std())
        plt.show()

В целом, это уже похоже на распределение плотности. Значит разумный размер burnin -- 10^6